notebook used to get FID scores for models

In [22]:

#? resources
#? vid 1: https://www.youtube.com/watch?v=HoKDTa5jHvg&t=177s
# explains how ddpm works
#? vid2: https://www.youtube.com/watch?v=TBCRlnwJtZU
# goes over implementation to ddpm

import random
import os
import copy
import numpy as np
import torch
import torch.nn as nn
from matplotlib import pyplot as plt
from tqdm import tqdm
from torch import optim
from helper_utils import *

from modules import UNet, UNet_conditional, EMA
import logging 
from torch.utils.tensorboard import SummaryWriter
#torch.set_default_tensor_type('torch.cuda.FloatTensor')

# library for quantization
try:
    import bitsandbytes as bnb
    print('imported bitsandbytes')
    
except:
    print('cant import bitsandbytes')
    bnb = None

logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", level=logging.INFO, datefmt="%I:%M:%S")

class Diffusion:
    def __init__(self, noise_steps=1000, beta_start=1e-4, beta_end=0.02, img_size=256, device="cuda", USE_GPU = True):
        self.noise_steps = noise_steps
        self.beta_start = beta_start
        self.beta_end = beta_end
        self.img_size = img_size #resoultion of image #: side note from video --> for higher resolutions, training seperate upsamplers instead of training on bigger resolution images
        self.device = device
        
        if USE_GPU and torch.cuda.is_available():
            print("CUDAAAAAAAAAAA")
            self.use_cuda = torch.device('cuda')
        else:
            self.use_cuda = torch.device('cpu')
        
        #? right now using simple beta schedule --> open AI using cosine scheduler        
        self.beta = self.prepare_noise_schedule().to(device)
        self.alpha = 1. - self.beta
        self.alpha_hat = torch.cumprod(self.alpha, dim=0)
        
        #! try implementing cosine scheduler
        
    def prepare_noise_schedule(self):
        #? Creates a one-dimensional tensor of size steps whose values are evenly spaced from start to end, inclusive
        return torch.linspace(self.beta_start, self.beta_end, self.noise_steps)
    
    def noise_images(self, x, t):
        """Adds noise to image. You can iteratively add noise to image but vid 1 showed 
        a simplification that adds noise in 1 step. Which is this implementation
        Args:
            x (_type_): _description_
            t (_type_): _description_

        Returns:
            _type_: returns image with noise added on
        """
        sqrt_alpha_hat = torch.sqrt(self.alpha_hat[t])[:, None, None, None]
        sqrt_one_minus_alpha_hat = torch.sqrt(1 - self.alpha_hat[t])[:, None, None, None]
        E = torch.randn_like(x)
        return sqrt_alpha_hat * x + sqrt_one_minus_alpha_hat * E, E
    
    def sample_timesteps(self, n):
        """_summary_

        Args:
            n (_type_): _description_

        Returns:
            _type_: _description_
        """
        #? needed for algorithm for training
        return torch.randint(low=1, high=self.noise_steps, size=(n,))
    

    def sample(self, model, n, labels, channels=3, cfg_scale=3):
        """implements algorithm 2 from the ddpm paper in vid 1

        Args:
            model (_type_): _description_
            n (int): number of images we want to sample 

        Returns:
            _type_: _description_
        """
        logging.info(f"Sampling {n} new images....")
        #? see here for why we set model.eval() https://stackoverflow.com/questions/60018578/what-does-model-eval-do-in-pytorch
        #? essentially disables some some parts of torch for specific steps
        logging.info(f"Sampling {n} new images....")
        model.eval()
        with torch.no_grad():
            #? create initial images by sampling over normal dist (step 1)
            x = torch.randn((n, channels, self.img_size, self.img_size)).to(self.device)
            
            #? step 2, 3, 4
            for i in tqdm(reversed(range(1, self.noise_steps)), position=0):
                t = (torch.ones(n) * i).long().to(self.device) #? tensor of timestep
                predicted_noise = model(x, t, labels) #? feed that into model w/ current images
                
                #? noise
                if cfg_scale > 0:
                    uncond_predicted_noise = model(x, t, None)
                    predicted_noise = torch.lerp(uncond_predicted_noise, predicted_noise, cfg_scale)
                alpha = self.alpha[t][:, None, None, None]
                alpha_hat = self.alpha_hat[t][:, None, None, None]
                beta = self.beta[t][:, None, None, None]
                
                #? only want noise for timestemps greater than 1. done so b/c in last iteration, would make final outcome worse due to adding noise to finalized pixels
                if i > 1:
                    noise = torch.randn_like(x)
                else:
                    noise = torch.zeros_like(x)
                    
                #? alter image by removed a little bit of noise
                x = 1 / torch.sqrt(alpha) * (x - ((1 - alpha) / (torch.sqrt(1 - alpha_hat))) * predicted_noise) + torch.sqrt(beta) * noise
        
        #? switch back to train    
        model.train()
        x = (x.clamp(-1, 1) + 1) / 2 #? brings back value to 0-1 range 
        x = (x * 255).type(torch.uint8) #? bring back values to pixel range for viewing image
        return x


===================================BUG REPORT===================================
The following directories listed in your path were found to be non-existent: {WindowsPath('C'), WindowsPath('/Users/Efran/anaconda3/envs/compsci682/lib')}
The following directories listed in your path were found to be non-existent: {WindowsPath('vs/workbench/api/node/extensionHostProcess')}
The following directories listed in your path were found to be non-existent: {WindowsPath('/matplotlib_inline.backend_inline'), WindowsPath('module')}
CUDA_SETUP: WARNING! libcudart.so not found in any environmental path. Searching in backup paths...
The following directories listed in your path were found to be non-existent: {WindowsPath('/usr/local/cuda/lib64')}
DEBUG: Possible options found for libcudart.so: set()
CUDA SETUP: PyTorch settings found: CUDA_VERSION=118, Highest Compute Capability: 8.9.
CUDA SETUP: To manually override the PyTorch CUDA version please see:https://github.com/TimDettmers/bitsandbytes/blob/

c:\Users\Efran\anaconda3\envs\compsci682\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes


  warn(msg)
c:\Users\Efran\anaconda3\envs\compsci682\lib\site-packages\bitsandbytes\cuda_setup\main.py:166: UserWarning: C:\Users\Efran\anaconda3\envs\compsci682 did not contain ['libcudart.so', 'libcudart.so.11.0', 'libcudart.so.12.0'] as expected! Searching further paths...
  warn(msg)


### FID Score

In [23]:
#? load mnist data set
import argparse
parser = argparse.ArgumentParser()
args = parser.parse_args(args=[])
args.batch_size = 16
args.dataset_path = r"mnist"
args.image_size = 56
args.channels = 1
dataloader = get_data(args)

def get_fid_score(path, mnist, model_names = ('49_ckpt.pt', '49_ema_ckpt.pt')):
    mnist_dataset = mnist.dataset
    
    
    #? load models
    model_path = path
    loaded_unet_model = torch.load(os.path.join(model_path, model_names[0]))
    loaded_unet_model.eval()

    loaded_ema_model = torch.load(os.path.join(model_path, model_names[1]))
    loaded_ema_model.eval()

    diffusion = Diffusion(img_size=args.image_size, device='cuda')
    print('loaded models')

    #? load FID
    from torchmetrics.image.fid import FrechetInceptionDistance

    fid = FrechetInceptionDistance(normalize=True)
    fid_batch_size = 16
    
    
    num_list = [random.randint(0, 9) for x in range(fid_batch_size)]
    labels = torch.tensor(num_list).long().to('cuda')
    print(labels, len(labels))

    #? create sample images for conditional
    sampled_images = diffusion.sample(loaded_unet_model, n=len(labels), channels=1, labels=labels)
    
    real_images = torch.stack([mnist_dataset[i][0] for i in range(fid_batch_size)],0)
    print(real_images.shape)

    fake_images = torch.stack([sampled_images, sampled_images, sampled_images], dim=1).reshape(fid_batch_size,3,56,56)
    real_images = torch.stack([real_images, real_images, real_images], dim=1).reshape(fid_batch_size,3,56,56)
    
    fake_images = fake_images.to("cpu")
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    conditional_fid = fid.compute()

    print('conditional fid:', conditional_fid)
    
    ema_sampled_images = diffusion.sample(loaded_ema_model, n=len(labels), channels=1, labels=labels)
    fid = FrechetInceptionDistance(normalize=True)
    fake_images = torch.stack([ema_sampled_images, ema_sampled_images, ema_sampled_images], dim=1).reshape(fid_batch_size,3,56,56)

    fake_images = fake_images.to("cpu")
    fid.update(real_images, real=True)
    fid.update(fake_images, real=False)
    fid.compute()

    conditional__ema_fid = fid.compute()

    print('conditional ema fid:', conditional__ema_fid)
    
    return conditional_fid, conditional__ema_fid


using mnist dataset
torch.Size([60000, 28, 28])
loaded mnist training dataset
Size of his data set:  60000


#### base model + deepwise model fid

In [24]:
dataloader = get_data(args)
model_path = r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\base_mnist_ddpm_conditional_ema'

results = get_fid_score(model_path, dataloader)
print(results)

using mnist dataset
torch.Size([60000, 28, 28])
loaded mnist training dataset
Size of his data set:  60000
CUDAAAAAAAAAAA
loaded models


12:33:33 - INFO: Sampling 16 new images....
12:33:33 - INFO: Sampling 16 new images....


tensor([9, 0, 4, 2, 4, 9, 1, 3, 6, 3, 8, 7, 3, 4, 9, 0], device='cuda:0') 16


999it [01:21, 12.30it/s]


torch.Size([16, 1, 56, 56])


12:34:58 - INFO: Sampling 16 new images....
12:34:58 - INFO: Sampling 16 new images....


conditional fid: tensor(284.2845)


999it [01:19, 12.50it/s]


conditional ema fid: tensor(267.2341)
(tensor(284.2845), tensor(267.2341))


In [25]:
model_path = r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\d_2_mnist_ddpm_conditional_ema'

results = get_fid_score(model_path, dataloader)
print(results)

CUDAAAAAAAAAAA
loaded models


05:08:32 - INFO: Sampling 16 new images....
05:08:32 - INFO: Sampling 16 new images....


tensor([4, 9, 1, 4, 9, 6, 6, 3, 7, 2, 5, 3, 2, 3, 9, 8], device='cuda:0') 16


999it [01:22, 12.08it/s]


torch.Size([16, 1, 56, 56])


05:09:59 - INFO: Sampling 16 new images....
05:09:59 - INFO: Sampling 16 new images....


conditional fid: tensor(308.4738)


999it [01:26, 11.58it/s]


conditional ema fid: tensor(303.7385)
(tensor(308.4738), tensor(303.7385))


In [23]:
model_path = r'F:\Classes\COMPSCI 682\denoising-diffusion-pytorch-main\ddpm\models\d_4_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader)
print(results)

CUDAAAAAAAAAAA
loaded models


05:02:57 - INFO: Sampling 16 new images....
05:02:57 - INFO: Sampling 16 new images....


tensor([0, 6, 8, 7, 1, 1, 7, 6, 1, 4, 7, 7, 6, 7, 7, 2], device='cuda:0') 16


999it [01:20, 12.40it/s]


torch.Size([16, 1, 56, 56])


05:04:21 - INFO: Sampling 16 new images....
05:04:21 - INFO: Sampling 16 new images....


conditional fid: tensor(248.1439)


999it [01:20, 12.36it/s]


conditional ema fid: tensor(274.1627)
(tensor(248.1439), tensor(274.1627))


#### pruned l1_unstructured models fid

In [10]:
model_path = r'models\p_l1_unstructured_10_base_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


06:41:26 - INFO: Sampling 16 new images....
06:41:26 - INFO: Sampling 16 new images....


tensor([3, 4, 5, 3, 0, 9, 0, 5, 3, 9, 0, 9, 0, 6, 3, 6], device='cuda:0') 16


999it [01:20, 12.40it/s]


torch.Size([16, 1, 56, 56])


06:42:50 - INFO: Sampling 16 new images....
06:42:50 - INFO: Sampling 16 new images....


conditional fid: tensor(274.0095)


999it [01:19, 12.50it/s]


conditional ema fid: tensor(276.0634)
(tensor(274.0095), tensor(276.0634))


In [11]:
model_path = r'models\p_l1_unstructured_10_d_2_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


06:47:13 - INFO: Sampling 16 new images....
06:47:13 - INFO: Sampling 16 new images....


tensor([9, 0, 0, 4, 8, 1, 9, 1, 5, 4, 7, 4, 2, 7, 5, 1], device='cuda:0') 16


999it [01:21, 12.25it/s]


torch.Size([16, 1, 56, 56])


06:48:38 - INFO: Sampling 16 new images....
06:48:38 - INFO: Sampling 16 new images....


conditional fid: tensor(284.6199)


999it [01:21, 12.25it/s]


conditional ema fid: tensor(254.6614)
(tensor(284.6199), tensor(254.6614))


In [12]:
model_path = r'models\p_l1_unstructured_10_d_4_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


06:50:04 - INFO: Sampling 16 new images....
06:50:04 - INFO: Sampling 16 new images....


tensor([1, 3, 3, 5, 9, 4, 6, 8, 8, 9, 9, 3, 0, 6, 3, 1], device='cuda:0') 16


999it [01:21, 12.22it/s]


torch.Size([16, 1, 56, 56])


06:51:29 - INFO: Sampling 16 new images....
06:51:29 - INFO: Sampling 16 new images....


conditional fid: tensor(263.8107)


999it [01:24, 11.78it/s]


conditional ema fid: tensor(288.1787)
(tensor(263.8107), tensor(288.1787))


In [13]:
model_path = r'models\p_l1_unstructured_30_base_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


06:52:58 - INFO: Sampling 16 new images....


tensor([9, 0, 7, 9, 0, 4, 0, 7, 7, 8, 9, 0, 6, 7, 2, 4], device='cuda:0') 16


06:52:58 - INFO: Sampling 16 new images....
999it [01:26, 11.57it/s]


torch.Size([16, 1, 56, 56])


06:54:29 - INFO: Sampling 16 new images....
06:54:29 - INFO: Sampling 16 new images....


conditional fid: tensor(292.7821)


999it [01:24, 11.79it/s]


conditional ema fid: tensor(284.0872)
(tensor(292.7821), tensor(284.0872))


In [14]:
model_path = r'models\p_l1_unstructured_30_d_2_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA


06:55:58 - INFO: Sampling 16 new images....


loaded models
tensor([2, 9, 3, 2, 5, 0, 3, 1, 6, 6, 3, 0, 8, 0, 3, 4], device='cuda:0') 16


06:55:58 - INFO: Sampling 16 new images....
999it [01:24, 11.84it/s]


torch.Size([16, 1, 56, 56])


06:57:26 - INFO: Sampling 16 new images....
06:57:26 - INFO: Sampling 16 new images....


conditional fid: tensor(302.5300)


999it [01:25, 11.70it/s]


conditional ema fid: tensor(257.3767)
(tensor(302.5300), tensor(257.3767))


In [15]:
model_path = r'models\p_l1_unstructured_30_d_4_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


06:58:55 - INFO: Sampling 16 new images....


tensor([9, 7, 3, 2, 5, 1, 5, 5, 9, 3, 4, 3, 7, 5, 5, 5], device='cuda:0') 16


06:58:55 - INFO: Sampling 16 new images....
999it [01:22, 12.15it/s]


torch.Size([16, 1, 56, 56])


07:00:21 - INFO: Sampling 16 new images....
07:00:21 - INFO: Sampling 16 new images....


conditional fid: tensor(204.8231)


999it [01:22, 12.07it/s]


conditional ema fid: tensor(207.1623)
(tensor(204.8231), tensor(207.1623))


#### pruned random models fid

In [16]:
model_path = r'models\p_random_10_base_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


07:01:48 - INFO: Sampling 16 new images....
07:01:48 - INFO: Sampling 16 new images....


tensor([6, 6, 2, 7, 6, 6, 5, 4, 4, 7, 5, 3, 9, 3, 2, 9], device='cuda:0') 16


999it [01:22, 12.13it/s]


torch.Size([16, 1, 56, 56])


07:03:15 - INFO: Sampling 16 new images....
07:03:15 - INFO: Sampling 16 new images....


conditional fid: tensor(410.1934)


999it [01:22, 12.14it/s]


conditional ema fid: tensor(224.0613)
(tensor(410.1934), tensor(224.0613))


In [17]:
model_path = r'models\p_random_10_d_2_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


07:04:41 - INFO: Sampling 16 new images....
07:04:41 - INFO: Sampling 16 new images....


tensor([3, 8, 4, 7, 2, 8, 3, 1, 0, 2, 8, 6, 4, 4, 1, 9], device='cuda:0') 16


999it [01:23, 11.90it/s]


torch.Size([16, 1, 56, 56])


07:06:09 - INFO: Sampling 16 new images....
07:06:09 - INFO: Sampling 16 new images....


conditional fid: tensor(367.8468)


999it [01:24, 11.88it/s]


conditional ema fid: tensor(360.7809)
(tensor(367.8468), tensor(360.7809))


In [18]:
model_path = r'models\p_random_10_d_4_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


07:07:37 - INFO: Sampling 16 new images....
07:07:37 - INFO: Sampling 16 new images....


tensor([3, 2, 5, 4, 9, 6, 9, 0, 0, 5, 1, 9, 0, 9, 6, 3], device='cuda:0') 16


999it [01:22, 12.15it/s]


torch.Size([16, 1, 56, 56])


07:09:03 - INFO: Sampling 16 new images....
07:09:03 - INFO: Sampling 16 new images....


conditional fid: tensor(356.1158)


999it [01:22, 12.15it/s]


conditional ema fid: tensor(358.8737)
(tensor(356.1158), tensor(358.8737))


In [19]:
model_path = r'models\p_random_30_base_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


07:10:29 - INFO: Sampling 16 new images....
07:10:29 - INFO: Sampling 16 new images....


tensor([2, 3, 2, 4, 9, 2, 2, 0, 2, 9, 5, 8, 9, 9, 5, 9], device='cuda:0') 16


999it [01:19, 12.62it/s]


torch.Size([16, 1, 56, 56])


07:11:52 - INFO: Sampling 16 new images....
07:11:52 - INFO: Sampling 16 new images....


conditional fid: tensor(474.7346)


999it [01:21, 12.23it/s]


conditional ema fid: tensor(493.6962)
(tensor(474.7346), tensor(493.6962))


In [20]:
model_path = r'models\p_random_30_d_2_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

CUDAAAAAAAAAAA
loaded models


07:13:17 - INFO: Sampling 16 new images....


tensor([6, 3, 4, 8, 0, 3, 6, 6, 4, 4, 0, 4, 1, 7, 3, 3], device='cuda:0') 16


07:13:17 - INFO: Sampling 16 new images....
999it [01:25, 11.74it/s]


torch.Size([16, 1, 56, 56])


07:14:46 - INFO: Sampling 16 new images....
07:14:46 - INFO: Sampling 16 new images....


conditional fid: tensor(436.1765)


999it [01:24, 11.77it/s]


conditional ema fid: tensor(478.7686)
(tensor(436.1765), tensor(478.7686))


In [21]:
model_path = r'models\p_random_30_d_4_mnist_ddpm_conditional_ema'
results = get_fid_score(model_path, dataloader, ('pruned_49_ckpt.pt', 'pruned_49_ema_ckpt.pt'))
print(results)

07:16:15 - INFO: Sampling 16 new images....


CUDAAAAAAAAAAA
loaded models
tensor([2, 6, 0, 2, 3, 1, 5, 0, 8, 1, 4, 9, 1, 2, 2, 2], device='cuda:0') 16


07:16:15 - INFO: Sampling 16 new images....
999it [01:24, 11.87it/s]


torch.Size([16, 1, 56, 56])


07:17:44 - INFO: Sampling 16 new images....
07:17:44 - INFO: Sampling 16 new images....


conditional fid: tensor(462.5279)


999it [01:23, 11.89it/s]


conditional ema fid: tensor(439.1877)
(tensor(462.5279), tensor(439.1877))


In [ ]:
DDPM + CFG, Prune linear and conv2d globally 10%, Depthwise Convolutional Layers, 2 Groups
DDPM + CFG + EMA, Prune linear and conv2d globally 30%, Depthwise Convolutional Layers, 2 Groups
DDPM + CFG, Prune linear and conv2d globally 10%, Depthwise Convolutional Layers, 4 Groups
DDPM + CFG + EMA,Prune linear and conv2d globally 30%, Depthwise Convolutional Layers, 4 Groups



DDPM + CFG, Prune linear and conv2d globally 10%, Depthwise Convolutional Layers, 2 Groups
DDPM + CFG + EMA, Prune linear and conv2d globally 30%, Depthwise Convolutional Layers, 2 Groups
DDPM + CFG, Prune linear and conv2d globally 10%, Depthwise Convolutional Layers, 4 Groups
DDPM + CFG + EMA,Prune linear and conv2d globally 30%, Depthwise Convolutional Layers, 4 Groups